# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 2102, done.
remote: Counting objects: 100% (484/484), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 2102 (delta 308), reused 342 (delta 201), pack-reused 1618 (from 1)
Receiving objects: 100% (2102/2102), 144.72 MiB | 29.85 MiB/s, done.
Resolving deltas: 100% (1253/1253), done.
Updating files: 100% (238/238), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 26.9 MB/s eta 0:00:00


In [ ]:
%cd /kaggle/working/RECsys_Challenge2024
! python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [ ]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/Hybrid'

np.random.seed(42)

## Import the repository **RECsys_Challenge2024**

In [ ]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [8]:
config = {
    'model': 'ItemKNN_CFCBF_Hybrid',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_ItemKNN_CFCBF_Hybrid.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [ ]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [10]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [11]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [13]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [14]:
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

def objective_function_ItemKNN_CFCBF_Hybrid(optuna_trial):
    
    recommender_instance = ItemKNN_CFCBF_Hybrid_Recommender(URM_train, ICM_all)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 0, 750),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                   'feature_weighting': optuna_trial.suggest_categorical('feature_weighting', ["BM25", "TF-IDF", "none"]),
                   "ICM_weight": optuna_trial.suggest_float("ICM_weight", 0.1, 20.0, log=True)
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True     

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [15]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_ItemKNN_CFCBF_Hybrid, n_trials=100)

[I 2024-11-19 18:52:20,568] A new study created in RDB with name: hyperparameters_tuning_ItemKNN_CFCBF_Hybrid


Similarity column 38121 (100.0%), 627.52 column/sec. Elapsed time 1.01 min
EvaluatorHoldout: Processed 35595 (100.0%) in 55.19 sec. Users per second: 645


[I 2024-11-19 18:54:17,980] Trial 0 finished with value: 0.02207286224879953 and parameters: {'similarity': 'asymmetric', 'topK': 243, 'shrink': 21, 'feature_weighting': 'TF-IDF', 'ICM_weight': 0.3112772787354627, 'asymmetric_alpha': 1.0525560849843292}. Best is trial 0 with value: 0.02207286224879953.


Similarity column 26200 (68.7%), 87.00 column/sec. Elapsed time 5.02 min
Similarity column 38121 (100.0%), 87.17 column/sec. Elapsed time 7.29 min
EvaluatorHoldout: Processed 35595 (100.0%) in 43.09 sec. Users per second: 826


[I 2024-11-19 19:02:19,996] Trial 1 finished with value: 0.033098978365517805 and parameters: {'similarity': 'euclidean', 'topK': 582, 'shrink': 364, 'feature_weighting': 'BM25', 'ICM_weight': 0.44284718944886264, 'normalize_avg_row': True, 'similarity_from_distance_mode': 'log', 'normalize': False}. Best is trial 1 with value: 0.033098978365517805.


...

[I 2024-11-19 22:17:41,484] Trial 97 finished with value: 0.04759566284724133 and parameters: {'similarity': 'cosine', 'topK': 41, 'shrink': 398, 'feature_weighting': 'BM25', 'ICM_weight': 0.17044288498786464}. Best is trial 28 with value: 0.051176340087000304.


Similarity column 38121 (100.0%), 634.61 column/sec. Elapsed time 1.00 min
EvaluatorHoldout: Processed 35595 (100.0%) in 40.08 sec. Users per second: 888


[I 2024-11-19 22:19:23,393] Trial 98 finished with value: 0.03498101659542734 and parameters: {'similarity': 'jaccard', 'topK': 57, 'shrink': 456, 'feature_weighting': 'BM25', 'ICM_weight': 0.14133022098751763}. Best is trial 28 with value: 0.051176340087000304.


Similarity column 38121 (100.0%), 627.35 column/sec. Elapsed time 1.01 min
EvaluatorHoldout: Processed 35595 (100.0%) in 48.16 sec. Users per second: 739


[I 2024-11-19 22:21:13,368] Trial 99 finished with value: 0.03611153140377611 and parameters: {'similarity': 'cosine', 'topK': 152, 'shrink': 493, 'feature_weighting': 'none', 'ICM_weight': 0.32870558303849406}. Best is trial 28 with value: 0.051176340087000304.


## Some optuna visualizations on recommender parameters

In [16]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [17]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [ ]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH + '/' + GH_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = ItemKNN_CFCBF_Hybrid_Recommender(URM_train + URM_validation, ICM_all)
    recommender_instance.fit(**best_params)

Similarity column 38121 (100.0%), 632.02 column/sec. Elapsed time 1.01 min


# Testing

Create the recommendations for the submission. 

In [ ]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}.csv')

Submission file saved as /kaggle/working/submission_ItemKNN_CFCBF_Hybrid.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [ ]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/ItemKNN_CFCBF_HybridRecommender/best_params_ItemKNN_CFCBF_Hybrid.json' created successfully.


Save the history of the tuned model.

In [ ]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/ItemKNN_CFCBF_HybridRecommender/history_ItemKNN_CFCBF_Hybrid.db' created successfully.


Save the best trained model and its submission.

In [ ]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Submission/submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/ItemKNN_CFCBF_HybridRecommender/Submission/submission_ItemKNN_CFCBF_Hybrid.csv' created successfully.
ItemKNN_CFCBF_HybridRecommender: Saving model in file '/kaggle/working/best_ItemKNN_CFCBF_Hybrid_tuned'
ItemKNN_CFCBF_HybridRecommender: Saving complete
File 'TrainedModels/ItemKNN_CFCBF_HybridRecommender/best_ItemKNN_CFCBF_Hybrid_tuned.zip' created successfully.
